<a href="https://colab.research.google.com/github/hasanzeynal/Tensorflow-Developer-Certificate/blob/main/01_Neural_Network_Regression_in_TensorFlow/neural_network_regression_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Regression with Neural Network in TensorFlow

In [1]:
#import tensorflow
import tensorflow as tf
print(tf.__version__)

2.12.0


# Basic Neural Network Model for Regression Problem(Car Price Prediction)
Then, After building our basic NN model, we are gonna try:
* improving our model
* evaluating
* setting
* saving
* loading
* processing with feature scalling(normalising, fitting model on a scaled data)

In [1]:
#ignore all warnings
import warnings
warnings.filterwarnings('ignore')
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
#load test and train dataset
test_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/test.csv')
train_df = pd.read_csv('/content/drive/MyDrive/datasets/Car_price/train.csv')

In [3]:
test_df.head()

,ID,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags,Price
0,44020629,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4,Manual,Front,02-Mar,Left wheel,Grey,10,NaN
1,45784798,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4,Tiptronic,Front,04-May,Left wheel,Grey,10,NaN
2,45800257,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4,Automatic,Front,04-May,Right-hand drive,Sky blue,8,NaN
3,45797981,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6,Automatic,4x4,04-May,Left wheel,Blue,12,NaN
4,45814303,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4,Manual,Front,04-May,Left wheel,Black,0,NaN


In [4]:
train_df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [5]:
#defining target variable
y = train_df['Price']

In [6]:
#drop the prices
test_df.drop('Price',axis=1,inplace=True)
train_df.drop('Price',axis=1,inplace=True)
#merge two df
X = test_df.append(train_df)
X.reset_index(inplace=True)
#drop id and index from X
X.drop(['ID','index'],axis=1,inplace=True)

In [7]:
X.head()

,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4.0,Manual,Front,02-Mar,Left wheel,Grey,10
1,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4.0,Tiptronic,Front,04-May,Left wheel,Grey,10
2,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4.0,Automatic,Front,04-May,Right-hand drive,Sky blue,8
3,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6.0,Automatic,4x4,04-May,Left wheel,Blue,12
4,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4.0,Manual,Front,04-May,Left wheel,Black,0


In [8]:
#checking for duplicate and null values
X.duplicated().sum(),X.isnull().any().sum()

(6253, 0)

In [9]:
#create new dataframe to show number of unique values in the dataset
inform = pd.DataFrame()
inform['Type of Column'] = X.dtypes
inform['Number of Uniques'] = X.nunique()
inform

,Type of Column,Number of Uniques
Levy,object,635
Manufacturer,object,68
Model,object,1981
Prod. year,int64,59
Category,object,11
Leather interior,object,2
Fuel type,object,7
Engine volume,object,112
Mileage,object,9977
Cylinders,float64,16


In [10]:
#change the column names
X.columns = [i.lower() for i in X.columns]
#replace all space with -
X.columns = [i.replace(' ','_') for i in X.columns]
X.columns

Index(['levy', 'manufacturer', 'model', 'prod._year', 'category',
       'leather_interior', 'fuel_type', 'engine_volume', 'mileage',
       'cylinders', 'gear_box_type', 'drive_wheels', 'doors', 'wheel', 'color',
       'airbags'],
      dtype='object')

In [11]:
X.head()

,levy,manufacturer,model,prod._year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags
0,-,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0 Turbo,0 km,4.0,Manual,Front,02-Mar,Left wheel,Grey,10
1,-,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000 km,4.0,Tiptronic,Front,04-May,Left wheel,Grey,10
2,-,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000 km,4.0,Automatic,Front,04-May,Right-hand drive,Sky blue,8
3,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000 km,6.0,Automatic,4x4,04-May,Left wheel,Blue,12
4,-,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000 km,4.0,Manual,Front,04-May,Left wheel,Black,0


In [12]:
# delete Trubo from engine volume
X['engine_volume'] = [i.split(' ')[0] for i in X['engine_volume']]
# delete km from milage
X['mileage'] = [i.split(' ')[0] for i in X['mileage']]
# change Mar and May from doors
X['doors'] = X['doors'].replace({'02-Mar':2,'04-May':4})
# replace - with 0 in the levy column
X['levy'] = X['levy'].replace({'-':0})

In [13]:
X.head()

,levy,manufacturer,model,prod._year,category,leather_interior,fuel_type,engine_volume,mileage,cylinders,gear_box_type,drive_wheels,doors,wheel,color,airbags
0,0,VOLKSWAGEN,Golf,2012,Hatchback,No,Diesel,2.0,0,4.0,Manual,Front,2,Left wheel,Grey,10
1,0,HYUNDAI,Sonata,2012,Sedan,Yes,Petrol,2.4,26000,4.0,Tiptronic,Front,4,Left wheel,Grey,10
2,0,NISSAN,Tiida,2005,Sedan,No,Petrol,1.5,168000,4.0,Automatic,Front,4,Right-hand drive,Sky blue,8
3,975,VOLVO,XC90,2012,Jeep,Yes,Petrol,3.2,143000,6.0,Automatic,4x4,4,Left wheel,Blue,12
4,0,OPEL,Astra,1993,Hatchback,No,Petrol,1.6,200000,4.0,Manual,Front,4,Left wheel,Black,0
